### DEM download and precessing
#### 1. srtm dem downloading; 
#### 2. srtm dem processing, including dem mosaic and downsampling.


In [58]:
import os
import rasterio as rio
import geopandas as gpd
from glob import glob
from utils.get_dem import get_dem
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling


In [59]:
path_hma_gtng = 'data/hma-extent/HMA/gtng_202307_hma_subregions.gpkg'
hma_vec_gdf = gpd.read_file(path_hma_gtng)
hma_vec_gdf.head()


,o1region,o2region,full_name,long_code,geometry
0,13,13-01,Hissar Alay,13-01_hissar_alay,"MULTIPOLYGON (((70 40.7, 71 40.7, 72.01 40.7, ..."
1,13,13-02,Pamir (Safed Khirs / West Tarim),13-02_pamir_safed_khirs_west_tarim,"MULTIPOLYGON (((74.35547 39.80418, 74.37581 39..."
2,13,13-03,West Tien Shan,13-03_west_tien_shan,"MULTIPOLYGON (((77.99937 43.69747, 78.5 43.749..."
3,13,13-04,East Tien Shan (Dzhungaria),13-04_east_tien_shan_dzhungaria,"MULTIPOLYGON (((78.5 43.74989, 78.5 44, 78.5 4..."
4,13,13-05,West Kun Lun,13-05_west_kun_lun,"MULTIPOLYGON (((76.49466 37.98237, 76.50852 37..."


#### **Download SRTM DEM**


In [3]:
# hma_bounds = ([75, 76, 34, 35])  ### west, east, south, north
lonmin, lonmax, latmin, latmax = 60, 110, 20, 50
for lon in range(lonmin, lonmax, 5):    
    for lat in range(latmin, latmax, 5):        
        dem_out = 'dem/tiles/SRTMGL3_{}_{}.tif'.format(lon, lat)
        region = [lon-0.1, lon+5+0.1, lat-0.1, lat+5+0.1]
        print('Ouput dem:', dem_out, 'Region:',  region)
        # get_dem(demtype='SRTMGL3', bounds=region, path_out=dem_out)


Ouput dem: dem/tiles/SRTMGL3_60_20.tif Region: [59.9, 65.1, 19.9, 25.1]
Ouput dem: dem/tiles/SRTMGL3_60_25.tif Region: [59.9, 65.1, 24.9, 30.1]
Ouput dem: dem/tiles/SRTMGL3_60_30.tif Region: [59.9, 65.1, 29.9, 35.1]
Ouput dem: dem/tiles/SRTMGL3_60_35.tif Region: [59.9, 65.1, 34.9, 40.1]
Ouput dem: dem/tiles/SRTMGL3_60_40.tif Region: [59.9, 65.1, 39.9, 45.1]
Ouput dem: dem/tiles/SRTMGL3_60_45.tif Region: [59.9, 65.1, 44.9, 50.1]
Ouput dem: dem/tiles/SRTMGL3_65_20.tif Region: [64.9, 70.1, 19.9, 25.1]
Ouput dem: dem/tiles/SRTMGL3_65_25.tif Region: [64.9, 70.1, 24.9, 30.1]
Ouput dem: dem/tiles/SRTMGL3_65_30.tif Region: [64.9, 70.1, 29.9, 35.1]
Ouput dem: dem/tiles/SRTMGL3_65_35.tif Region: [64.9, 70.1, 34.9, 40.1]
Ouput dem: dem/tiles/SRTMGL3_65_40.tif Region: [64.9, 70.1, 39.9, 45.1]
Ouput dem: dem/tiles/SRTMGL3_65_45.tif Region: [64.9, 70.1, 44.9, 50.1]
Ouput dem: dem/tiles/SRTMGL3_70_20.tif Region: [69.9, 75.1, 19.9, 25.1]
Ouput dem: dem/tiles/SRTMGL3_70_25.tif Region: [69.9, 75.1, 24.9

### Mosaic

In [4]:
paths_dem_ls = glob('data/dem/tiles/*')
path_mosaic = 'data/dem/hma_SRTMGL3_90m.tif'

src_files_to_mosaic = []
for fp in paths_dem_ls:
    src = rio.open(fp)
    src_files_to_mosaic.append(src)
mosaic_arr, mosaic_trans = merge(src_files_to_mosaic)
mosaic_meta = src.meta.copy()
mosaic_meta.update({
    "height": mosaic_arr.shape[1],
    "width": mosaic_arr.shape[2],
    "transform": mosaic_trans
    })

# Write the mosaic raster to disk
with rio.open(path_mosaic, 'w', **mosaic_meta) as dest:
    dest.write(mosaic_arr)


### Downsampling

In [5]:
path_srtm = 'data/dem/hma_SRTMGL3_90m.tif'
path_srtm_down = 'data/dem/hma_SRTMGL3_500m.tif' 

with rio.open(path_srtm) as src:
    scale_factor = 500 / 90  
    dst_width, dst_height = int(src.width / scale_factor), int(src.height / scale_factor)    
    dst_transform = src.transform * src.transform.scale(scale_factor, scale_factor)
    dst_meta = src.meta.copy()
    dst_meta.update({
        "width": dst_width,
        "height": dst_height,
        "transform": dst_transform})
    with rio.open(path_srtm_down, "w", **dst_meta) as dst:
        reproject(
            source=rio.band(src, 1),
            destination=rio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_transform,
            dst_crs=src.crs,
            resampling=Resampling.average,  
        )


#### clip to hma subregions. 

In [67]:
path_dem = 'data/dem/hma_SRTMGL3_90m.tif'
with rio.open(path_dem) as src:
    if hma_vec_gdf.crs != src.crs: 
        hma_vec_gdf = hma_vec_gdf.to_crs(src.crs)      
    for idx, row in hma_vec_gdf.iterrows():
        geom = row.geometry
        print(geom.bounds)
        clipped_arr, clipped_transform = mask(dataset=src, shapes = [geom], 
                                              crop=True, all_touched=True)
        meta = src.meta.copy()
        meta.update({
            "height": clipped_arr.shape[1],
            "width": clipped_arr.shape[2],
            "transform": clipped_transform})
        ## save to path
        name_subregion = row['full_name'].split(' (')[0].replace(' ', '_').lower()
        path_save = f"data/dem/hma-subregions/dem_90m_{name_subregion}.tif"
        if os.path.exists(path_save): os.remove(path_save)
        with rio.open(path_save, "w", **meta) as dst:
            print(dst.bounds)
            dst.write(clipped_arr)
        print(f"saved to: {path_save}")
        break


(67.0, 38.0, 74.65889614, 40.7334286100001)
BoundingBox(left=66.99958333327722, bottom=37.99958333333833, right=74.65958333327548, top=40.73375000000438)
saved to: data/dem/hma-subregions/dem_90m_hissar_alay.tif


In [50]:
# # # ## downsampling
# path_srtm = 'dem/hma_SRTMGL3.tif'
# path_srtm_down = 'dem/hma_SRTMGL3_300m.tif' 
# !gdal_translate -outsize 30% 30% -r average $path_srtm $path_srtm_down


### Dem transform
#### Shaded relief map


In [51]:
# ## Generate a shaded relief map 
# path_dem_shade_relif = 'dem/hma_srtmgl3_down_shade_relif.tif'
# path_dem = 'dem/hma_SRTMGL3_down.tif'
# !gdaldem hillshade $path_dem $path_dem_shade_relif



### Check the dem image.

In [52]:
# ## Check
# srtm_down = readTiff(path_srtm_down)
# cmap = plt.cm.terrain
# plt.figure(figsize=(8, 7))
# plt.subplot(1,1,1)
# plt.title('downsampled SRTM data')
# plt.imshow(srtm_down.array, cmap=cmap, clim=[0, 8000], alpha=1, extent=srtm_down.geoextent)
# plt.colorbar(fraction=0.0320, pad=0.02, label='Elevation (m)', shrink=0.65)



In [53]:
# ## check shade relif map
# srtm_down_shade_relif = readTiff(path_dem_shade_relif)
# cmap = plt.cm.terrain
# plt.figure(figsize=(8, 7))
# plt.subplot(1,1,1)
# plt.title('downsampled SRTM shaded relif map')
# plt.imshow(srtm_down_shade_relif.array, cmap=cmap, \
#                   clim=[0, 200], alpha=1, extent=srtm_down_shade_relif.geoextent)
# plt.colorbar(fraction=0.0320, pad=0.02, label='Elevation (m)')
